## Exercises


### Decision Tree

Using the titanic data, in your classification-exercises repository, create a notebook, model.ipynb where you will do the following:

In [1]:
import numpy as np
import pandas as pd
from io import StringIO
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text


import acquire
import prepare



from sklearn.metrics import confusion_matrix
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')



In [2]:
# Pull in the Data

In [2]:
# acquire the data
train, validate, test = prepare.prep_titanic(acquire.get_titanic_data()) #Dataframe features and target

In [3]:
# look the shape.
train.shape, validate.shape, test.shape

((497, 14), (214, 14), (178, 14))

In [5]:
# creating 

In [4]:
#created df features excluding our target survived.
X_train, y_train = train.drop(columns=['survived']), train.survived #DataFrame Features, target
X_validate, y_validate = validate.drop(columns=['survived']), validate.survived #DataFrame Features, Target
X_test, y_test = test.drop(columns=['survived']), test.survived

### 1.)What is your baseline prediction? What is your baseline accuracy? remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). When you make those predictions, what is your accuracy? This is your baseline accuracy.





For a classification problem, a common choice for the baseline model is a model that simply predicts the most common class every single time.

### Model 1

In [5]:
# baseline being the most fequently occuring in our target.
train.survived.value_counts()

0    307
1    190
Name: survived, dtype: int64

In [8]:
# The baseline prediction is going to be died (0)

In [6]:
train['surv_baseline_pred'] = 0

In [7]:
train.head(2)

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,embark_town,alone,Q,S,surv_baseline_pred
474,474,0,3,female,22.0,0,0,9.8375,S,Third,Southampton,1,0,1,0
167,167,0,3,female,45.0,1,4,27.9000,S,Third,Southampton,0,0,1,0


In [10]:
baseline_accuracy = (train.surv_baseline_pred == train.survived).mean()
baseline_accuracy

0.6177062374245473

### 2.)Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)



In [11]:
#fit our thing

In [8]:
X_train, y_train = train.drop(columns=['class', 'embark_town', 'embarked', 'surv_baseline_pred']), train.survived 
X_validate, y_validate = validate.drop(columns=['class', 'embark_town', 'embarked']), validate.survived 
X_test, y_test = test.drop(columns=['class', 'embark_town', 'embarked']), test.survived

In [9]:
# Create X_train which will exclude our target!
X_train.head(2)

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,alone,Q,S
474,474,0,3,female,22.0,0,0,9.8375,1,0,1
167,167,0,3,female,45.0,1,4,27.9000,0,0,1


In [13]:
#Incase want to change sex to numeric
X_train.sex[X_train.sex == 'female'] = 0
X_train.sex[X_train.sex == 'male'] = 1

In [12]:
X_validate.sex[X_validate.sex == 'female'] = 0
X_validate.sex[X_validate.sex == 'male'] = 1

In [16]:
X_test.sex[X_test.sex == 'female'] = 0
X_test.sex[X_test.sex == 'male'] = 1

In [14]:
X_validate.head(3)

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,alone,Q,S
457,457,1,1,0,30.074375,1,0,51.8625,0,0,1
121,121,0,3,1,30.074375,0,0,8.0500,1,0,1
886,886,0,2,1,27.000000,0,0,13.0000,1,0,1


In [51]:
model1 = DecisionTreeClassifier(max_depth = 10)
model1.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=10)

### 3.)Evaluate your in-sample results using the model score, confusion matrix, and classification report.



In [52]:
#model score, Inmodel!!!
train_score = print(f'training score : {model1.score(X_train, y_train):.2%}')
train_score

training score : 91.95%


In [53]:
#model score, Inmodel!!!
validate_score = model1.score(X_validate, y_validate)
validate_score

0.6915887850467289

In [54]:
test_score = model1.score(X_test, y_test)
test_score

0.6460674157303371

In [55]:
train['model1_prediction'] = model1.predict(X_train)


In [23]:
train.head(3)

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,embark_town,alone,Q,S,surv_baseline_pred,model1_prediction
474,474,0,3,female,22.0,0,0,9.8375,S,Third,Southampton,1,0,1,0,0
167,167,0,3,female,45.0,1,4,27.9000,S,Third,Southampton,0,0,1,0,0
271,271,1,3,male,25.0,0,0,0.0000,S,Third,Southampton,1,0,1,0,1


In [23]:
#Create a Confusion Matrix

In [19]:
pred_df = pd.DataFrame([['TN', 'FP'],['FN', 'TP']], index=['actual death', 'actual survived'], columns=['pred_2 death', 'pred_2 survived'])

pred_df + ' : ' + confusion_matrix(train.survived, train.model1_prediction).astype(str)



,pred_2 death,pred_2 survived
actual death,TN : 307,FP : 0
actual survived,FN : 0,TP : 190


In [24]:
# classification report
pd.DataFrame(classification_report(train.survived, train.model1_prediction, output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,1.0,1.0,1.0,1.0,1.0
recall,1.0,1.0,1.0,1.0,1.0
f1-score,1.0,1.0,1.0,1.0,1.0
support,307.0,190.0,1.0,497.0,497.0


In [25]:
validate.head(2)

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,embark_town,alone,Q,S
457,457,1,1,female,30.074375,1,0,51.8625,S,First,Southampton,0,0,1
121,121,0,3,male,30.074375,0,0,8.0500,S,Third,Southampton,1,0,1


### 4.)Compute: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.



In [26]:
pd.DataFrame(classification_report(train.survived,train.model1_prediction, output_dict = True))

,0,1,accuracy,macro avg,weighted avg
precision,1.0,1.0,1.0,1.0,1.0
recall,1.0,1.0,1.0,1.0,1.0
f1-score,1.0,1.0,1.0,1.0,1.0
support,307.0,190.0,1.0,497.0,497.0


In [28]:
pred_df = pd.DataFrame([['TN', 'FP'],['FN', 'TP']], index=['actual death', 'actual survived'], columns=['pred_2 death', 'pred_2 survived'])

pred_df + ' : ' + confusion_matrix(train.survived, train.model1_prediction).astype(str)

,pred_2 death,pred_2 survived
actual death,TN : 307,FP : 0
actual survived,FN : 0,TP : 190


In [30]:
TN= 307
FN= 0
FP= 0
TP= 190

tpr = TP/(TP+FN)
fpr = FP/(FP+TN)
tnr = TN/(TN+FP)
fnr = FN/(FN+TP)

tpr, fpr, tnr, fnr

(1.0, 0.0, 1.0, 0.0)

### 5.)Run through steps 2-4 using a different max_depth value.



### Model 2

In [30]:
#Pull in Data!!

In [31]:
train, validate, test = prepare.prep_titanic(acquire.get_titanic_data())

In [34]:
#create train test and validate.

In [32]:
train.head(1)

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,embark_town,alone,Q,S
474,474,0,3,female,22.0,0,0,9.8375,S,Third,Southampton,1,0,1


In [ ]:
# we dont need embarked, class, embark_town, sex. On sex we can make numeric if we wanrt!!!

In [33]:
# Figure the baseline prediction.
train.survived.value_counts() 

0    307
1    190
Name: survived, dtype: int64

In [34]:
# need to create baseline accuracy!!!
train['baseline_prediction'] = 0

In [35]:
baseline_accuracy = (train.baseline_prediction == train.survived).mean()
baseline_accuracy

0.6177062374245473

In [36]:
# create out df features and target.
X_train, y_train = train.drop(columns=['survived','baseline_prediction','embarked', 'embark_town','class', 'sex']), train.survived 
X_validate, y_validate = validate.drop(columns=['survived','embarked', 'embark_town','class', 'sex']), validate.survived 
X_test, y_test = test.drop(columns= ['survived','embarked', 'embark_town','class', 'sex']), test.survived

In [ ]:
#before we fit need to change sex (categorical) str value of male or female to numeric 0 or 1
#also we could just drop sex!!!!

In [115]:
X_train.sex[X_train.sex == 'female'] = 0
X_train.sex[X_train.sex == 'male'] = 1


In [116]:
X_validate.sex[X_validate.sex == 'female'] = 0
X_validate.sex[X_validate.sex == 'male'] = 1

In [139]:
X_test.sex[X_test.sex == 'female'] = 0
X_test.sex[X_test.sex == 'male'] = 1

In [37]:
X_train.head(1)

,passenger_id,pclass,age,sibsp,parch,fare,alone,Q,S
474,474,3,22.0,0,0,9.8375,1,0,1


In [38]:
X_validate.head(1)

,passenger_id,pclass,age,sibsp,parch,fare,alone,Q,S
457,457,1,30.074375,1,0,51.8625,0,0,1


In [39]:
X_test.head(1)

,passenger_id,pclass,age,sibsp,parch,fare,alone,Q,S
769,769,3,32.0,0,0,8.3625,1,0,1


In [40]:
#Now create our thing!!!
model2 = DecisionTreeClassifier(max_depth = 5)

#We now fit out thing!!
model2.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=5)

In [41]:
train['prediction'] = model2.predict(X_train)

In [ ]:
#Next use our thing!! Create modelscore, confusion Matrix, and run a classification report.

In [42]:
#Confusion Matix
confusion_matrix(train.survived, train.prediction)

array([[279,  28],
       [ 72, 118]])

In [ ]:
# need to make out table readable so 

In [43]:
print(f'training score: {model2.score(X_train, y_train):.2%}')
print(f'validate score: {model2.score(X_validate, y_validate):.2%}')
print(f'test score: {model2.score(X_test, y_test):.2%}')

training score: 79.88%
validate score: 68.22%
test score: 70.22%


In [46]:
X_validate.head(3)

,passenger_id,pclass,age,sibsp,parch,fare,alone,Q,S
457,457,1,30.074375,1,0,51.8625,0,0,1
121,121,3,30.074375,0,0,8.0500,1,0,1
886,886,2,27.000000,0,0,13.0000,1,0,1


In [47]:
validate['model_pred'] = model2.predict(X_validate)

In [48]:
print(classification_report(train.survived, train.prediction))

              precision    recall  f1-score   support

           0       0.79      0.91      0.85       307
           1       0.81      0.62      0.70       190

    accuracy                           0.80       497
   macro avg       0.80      0.76      0.78       497
weighted avg       0.80      0.80      0.79       497



In [49]:
print(classification_report(validate.survived, validate.model_pred))

              precision    recall  f1-score   support

           0       0.72      0.79      0.75       132
           1       0.60      0.51      0.55        82

    accuracy                           0.68       214
   macro avg       0.66      0.65      0.65       214
weighted avg       0.68      0.68      0.68       214



In [50]:
pred_df = pd.DataFrame([['TN', 'FP'],['FN', 'TP']], index=['actual death', 'actual survived'], columns=['pred_2 death', 'pred_2 survived'])

pred_df + ' : ' + confusion_matrix(train.survived, train.prediction).astype(str)

,pred_2 death,pred_2 survived
actual death,TN : 279,FP : 28
actual survived,FN : 72,TP : 118


### 6.)Which model performs better on your in-sample data?



model 1 performs better wih an train accuracy: 95.37 and validate accuracy of 74% 

### 7.)Which model performs best on your out-of-sample data, the validate set?

In [ ]:
#model 2 performs best on out of sample data. Model 1 performs better on the validate set.

## Exercises Random Forest

Continue working in your model file with titanic data to do the following:



In [72]:
#Bring in our data and split it.
train, validate, test = prepare.prep_titanic(acquire.get_titanic_data())

In [24]:
#look at the shape of test,validate,test!!!

In [32]:
train.shape, validate.shape, test.shape

((497, 14), (214, 14), (178, 14))

In [75]:
train.survived.value_counts()

0    307
1    190
Name: survived, dtype: int64

In [76]:
train['surv_baseline_pred'] = 0

In [80]:
# create our baseline to compare back to!!!
baseline = (train.surv_baseline_pred == train.survived).mean()
baseline

0.6177062374245473

In [4]:
# Create X_train which will exclude our target!
X_train = train.drop(columns = ['class', 'embark_town', 'embarked', 'survived'])
X_train.head(5)

,passenger_id,pclass,sex,age,sibsp,parch,fare,alone,Q,S
583,583,1,male,36.000000,0,0,40.1250,1,0,0
337,337,1,female,41.000000,0,0,134.5000,1,0,0
50,50,3,male,7.000000,4,1,39.6875,0,0,1
218,218,1,female,32.000000,0,0,76.2917,1,0,0
31,31,1,female,29.916875,1,0,146.5208,0,0,0


In [5]:
X_validate = validate.drop(columns = ['class', 'embark_town', 'embarked', 'survived'])
X_validate.head(3)

,passenger_id,pclass,sex,age,sibsp,parch,fare,alone,Q,S
610,610,3,female,39.000000,1,5,31.2750,0,0,1
424,424,3,male,18.000000,1,1,20.2125,0,0,1
568,568,3,male,29.916875,0,0,7.2292,1,0,0


In [ ]:
#create X_validate from validate, excluding target

In [38]:
X_validate = validate.drop(columns = ['class', 'embark_town', 'embarked', 'survived'])
X_validate.head(3)

,passenger_id,pclass,sex,age,sibsp,parch,fare,alone,Q,S
610,610,3,female,39.000000,1,5,31.2750,0,0,1
424,424,3,male,18.000000,1,1,20.2125,0,0,1
568,568,3,male,29.916875,0,0,7.2292,1,0,0


In [73]:
#changing sex from categorical str to categ based on numeric on train dataset.
X_train.sex[X_train.sex == 'male'] = 0
X_train.sex[X_train.sex == 'female'] = 1

In [74]:
# changed sex from categorical str to categ based on numeric on validate dataset.  
X_validate.sex[X_validate.sex == 'male'] = 0
X_validate.sex[X_validate.sex == 'female'] = 1

In [35]:
#create our train target.
y_train = train['survived']
y_train.head()

583    0
337    1
50     0
218    1
31     1
Name: survived, dtype: int64

In [40]:
#create validate target
y_validate = validate['survived']

In [41]:
X_validate.head(1)

,passenger_id,pclass,sex,age,sibsp,parch,fare,alone,Q,S
610,610,3,1,39.0,1,5,31.275,0,0,1


### 1.) ### Fit the Random Forest classifier to your training sample and transform (i.e. make predictions on the training sample) setting the random_state accordingly and setting min_samples_leaf = 1 and max_depth = 10.

In [42]:
#make our thing
model = RandomForestClassifier(random_state = 1349,min_samples_leaf = 1, max_depth = 10)

In [44]:
#fit our thing
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, random_state=1349)

### 2.) Evaluate your results using the model score, confusion matrix, and classification report.

In [45]:
#Model Score:
print(f'training score: {model.score(X_train, y_train):.2%}')
print(f'validate score: {model.score(X_validate, y_validate):.2%}')

training score: 97.38%
validate score: 80.84%


In [48]:
y_pred = model.predict(X_train)

In [46]:
#add prediction to train to use confusion matrix
train['model_pred'] = model.predict(X_train)

In [59]:
validate['model_pred'] = model.predict(X_validate)

In [47]:
#confusion matrix 
matrix = confusion_matrix(train.survived, train.model_pred)
matrix

array([[307,   0],
       [ 13, 177]])

In [49]:
y_train.head()

583    0
337    1
50     0
218    1
31     1
Name: survived, dtype: int64

In [52]:
y_train = train['survived']

In [58]:
#Classification Report
pd.DataFrame(classification_report(train.survived, y_pred, output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.959375,1.000000,0.973843,0.979688,0.974906
recall,1.000000,0.931579,0.973843,0.965789,0.973843
f1-score,0.979266,0.964578,0.973843,0.971922,0.973651
support,307.000000,190.000000,0.973843,497.000000,497.000000


### 3.) Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [60]:
print(classification_report(validate.survived, validate.model_pred))

              precision    recall  f1-score   support

           0       0.84      0.86      0.85       132
           1       0.76      0.73      0.75        82

    accuracy                           0.81       214
   macro avg       0.80      0.79      0.80       214
weighted avg       0.81      0.81      0.81       214



### 4.) Run through steps increasing your min_samples_leaf and decreasing your max_depth.

In [63]:
## STEP 1:#######
#make our thing, decreased max_depth and increased min_samples_leaf
model1 = RandomForestClassifier(random_state = 1349,min_samples_leaf = 3, max_depth = 7)

In [64]:
##STEP 2:#######
#fit our thing
model1.fit(X_train, y_train)

RandomForestClassifier(max_depth=7, min_samples_leaf=3, random_state=1349)

In [65]:
###STEP 3: use our thing!!!
y_pred = model1.predict(X_train)

In [66]:
pd.DataFrame(classification_report(y_train, y_pred, output_dict= True))

,0,1,accuracy,macro avg,weighted avg
precision,0.888889,0.932927,0.903421,0.910908,0.905724
recall,0.964169,0.805263,0.903421,0.884716,0.903421
f1-score,0.925000,0.864407,0.903421,0.894703,0.901836
support,307.000000,190.000000,0.903421,497.000000,497.000000


### 5.) What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?

In [ ]:
# decrease in the accuracy,macro avg , weighted average. 